In [1]:
import numpy as np
from typing import Optional


class PCA:
    """
    Алгоритм анализа главных компонент (PCA) - техника понижения размерности.

    Parameters
    ----------
    n_components : int
        Количество главных компонент.
        Если n_components не задано, то все компоненты сохраняются.

    Attributes
    ----------
    mean_ : ndarray размера (n_features,)
        Эмпирическое среднее значение для каждого признака, рассчитанное на основе обучающей выборки.

    components_ : ndarray размера (n_features, n_components)
        Главные оси в пространстве признаков, представляющие направления максимальной дисперсии.

    explained_variance_ : ndarray размера (n_components,)
        Величина отклонения, охватываемая каждым из выбранных компонентов.

    explained_variance_ratio_ : ndarray размера (n_components,)
        Процент отклонения, объясняемый каждым из выбранных компонентов.
        Если параметр Of_components не задан, тогда сохраняются все компоненты, и сумма коэффициентов равна 1,0.
    """

    def __init__(self, n_components: Optional[int] = None):
        self.n_components = n_components

    def fit(self, X: np.ndarray):
        """
        Обучение модели на матрице X, выполнив разложение по ковариационной матрице.

        Parameters
        ----------
        X : ndarray размера (n_samples, n_features)
            Обучающая выборка.

        Returns
        -------
        self : object
            Возвращает сам объект.
        """
        # Расчет среднего
        self.mean_ = np.mean(X, axis=0)

        # Центрирование данных
        X = X - self.mean_

        # Расчет ковариационной матрицы
        n = X.shape[0]
        cov_matrix = np.dot(X.T, X) / (n - 1)

        # Расчет собственных значений и собствнных векторов
        eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

        # Сортировка собствнных векторов в порядке убывания
        idx = np.argsort(eigenvalues)[::-1]
        self.components_ = eigenvectors[:, idx]

        # Вычислить дисперсию
        self.explained_variance_ = eigenvalues[idx]
        self.explained_variance_ratio_ = self.explained_variance_ / np.sum(self.explained_variance_)

    def transform(self, X):
        """
        Применение уменьшения размерности к X.

        X проецируется на первые основные компоненты, ранее полученные из обучающего набора.

        Parameters
        ----------
        X : ndarray размера (n_samples, n_features)
            Новые данные.

        Returns
        -------
        X_new : ndarray размера (n_samples, n_components)
            Преобразованные значения.
        """
        # Центрирование данных
        X = X - self.mean_

        # Преобразованные данные
        return np.dot(X, self.components_[:, :self.n_components])

In [2]:
X = np.array(
    [
        [1.2, 2, 3.2],
        [4.2, 5.3, 6.6],
        [7, 8.1, 9.7],
        [9, 4, 4.5],
    ]
)

print("PCA with all components")
print("--------")
pca = PCA()
pca.fit(X)
X_projected = pca.transform(X)
print(pca.__dict__)
print(pca.explained_variance_ratio_.sum())
print(X.shape, X_projected.shape)
print(X_projected)
print("--------\n")

print("PCA with 1 component")
print("--------")
pca = PCA(1)
pca.fit(X)
X_projected = pca.transform(X)
print(pca.__dict__)
print(pca.explained_variance_ratio_.sum())
print(X.shape, X_projected.shape)
print(X_projected)
print("--------")

PCA with all components
--------
{'n_components': None, 'mean_': array([5.35, 4.85, 6.  ]), 'components_': array([[ 0.61498811,  0.78390513,  0.08533678],
       [ 0.54615757, -0.34539208, -0.76316199],
       [ 0.56877195, -0.51594288,  0.64054774]]), 'explained_variance_': array([1.91108305e+01, 7.00085066e+00, 1.65216263e-03]), 'explained_variance_ratio_': array([7.31841863e-01, 2.68094868e-01, 6.32689290e-05])}
0.9999999999999999
(4, 3) (4, 3)
[[-5.70131120e+00 -8.24198801e-01  2.73303455e-02]
 [-1.20202253e-01 -1.36648307e+00 -5.72315460e-02]
 [ 4.89419870e+00 -1.73806945e+00  3.05558775e-02]
 [ 9.27314752e-01  3.92875132e+00 -6.54677013e-04]]
--------

PCA with 1 component
--------
{'n_components': 1, 'mean_': array([5.35, 4.85, 6.  ]), 'components_': array([[ 0.61498811,  0.78390513,  0.08533678],
       [ 0.54615757, -0.34539208, -0.76316199],
       [ 0.56877195, -0.51594288,  0.64054774]]), 'explained_variance_': array([1.91108305e+01, 7.00085066e+00, 1.65216263e-03]), 'expla

In [4]:
class SVDPCA:
    """
    Алгоритм анализа главных компонент (PCA) - техника понижения размерности.

    Использование сингулярного разложения (SVD) для выполнения PCA.

    Parameters
    ----------
    n_components : int
        Количество главных компонент.
        Если n_components не задано, то все компоненты сохраняются.

    Attributes
    ----------
    mean_ : ndarray размера (n_features,)
        Эмпирическое среднее значение для каждого признака, рассчитанное на основе обучающей выборки.

    components_ : ndarray размера (n_features, n_components)
        Главные оси в пространстве признаков, представляющие направления максимальной дисперсии.

    explained_variance_ : ndarray размера (n_components,)
        Величина отклонения, охватываемая каждым из выбранных компонентов.

    explained_variance_ratio_ : ndarray размера (n_components,)
        Процент отклонения, объясняемый каждым из выбранных компонентов.
        Если параметр Of_components не задан, тогда сохраняются все компоненты, и сумма коэффициентов равна 1,0.
    """

    def __init__(self, n_components: Optional[int] = None):
        self.n_components = n_components

    def fit(self, X: np.ndarray) -> None:
        """
        Обучение модели на матрице X, выполнив разложение по ковариационной матрице.

        Parameters
        ----------
        X : ndarray размера (n_samples, n_features)
            Обучающая выборка.

        Returns
        -------
        self : object
            Возвращает сам объект.
        """
        # Расчет среднего
        self.mean_ = np.mean(X, axis=0)

        # Центрирование данных
        X = X - self.mean_

        # SVD разложение
        _, S, Vt = np.linalg.svd(X)

        # Выбор первых n_components
        self.components_ = Vt[:self.n_components].T

        # Вычислить дисперсию
        n = X.shape[0]
        self.explained_variance_ = (S ** 2) / (n - 1)
        self.explained_variance_ratio_ = self.explained_variance_ / np.sum(self.explained_variance_)

    def transform(self, X: np.ndarray) -> np.ndarray:
        """
        Применение уменьшения размерности к X.

        X проецируется на первые основные компоненты, ранее полученные из обучающего набора.

        Parameters
        ----------
        X : ndarray размера (n_samples, n_features)
            Новые данные.

        Returns
        -------
        X_new : ndarray размера (n_samples, n_components)
            Преобразованные значения.
        """
        # Центрирование данных
        X = X - self.mean_

        # Преобразованные данные
        return np.dot(X, self.components_[:, :self.n_components])  # type: ignore

In [5]:
X = np.array(
    [
        [1.2, 2, 3.2],
        [4.2, 5.3, 6.6],
        [7, 8.1, 9.7],
        [9, 4, 4.5],
    ]
)

print("PCA with all components")
print("--------")
pca = SVDPCA()
pca.fit(X)
X_projected = pca.transform(X)
print(pca.__dict__)
print(pca.explained_variance_ratio_.sum())
print(X.shape, X_projected.shape)
print(X_projected)
print("--------\n")

print("PCA with 1 component")
print("--------")
pca = SVDPCA(1)
pca.fit(X)
X_projected = pca.transform(X)
print(pca.__dict__)
print(pca.explained_variance_ratio_.sum())
print(X.shape, X_projected.shape)
print(X_projected)
print("--------")

PCA with all components
--------
{'n_components': None, 'mean_': array([5.35, 4.85, 6.  ]), 'components_': array([[ 0.61498811, -0.78390513, -0.08533678],
       [ 0.54615757,  0.34539208,  0.76316199],
       [ 0.56877195,  0.51594288, -0.64054774]]), 'explained_variance_': array([1.91108305e+01, 7.00085066e+00, 1.65216263e-03]), 'explained_variance_ratio_': array([7.31841863e-01, 2.68094868e-01, 6.32689290e-05])}
1.0
(4, 3) (4, 3)
[[-5.70131120e+00  8.24198801e-01 -2.73303455e-02]
 [-1.20202253e-01  1.36648307e+00  5.72315460e-02]
 [ 4.89419870e+00  1.73806945e+00 -3.05558775e-02]
 [ 9.27314752e-01 -3.92875132e+00  6.54677013e-04]]
--------

PCA with 1 component
--------
{'n_components': 1, 'mean_': array([5.35, 4.85, 6.  ]), 'components_': array([[0.61498811],
       [0.54615757],
       [0.56877195]]), 'explained_variance_': array([1.91108305e+01, 7.00085066e+00, 1.65216263e-03]), 'explained_variance_ratio_': array([7.31841863e-01, 2.68094868e-01, 6.32689290e-05])}
1.0
(4, 3) (4, 1

In [6]:
# import numpy as np
from PIL import Image
from PIL.Image import Image as PILImage


class ImagePCA:
    """
    Сжимает изображение с помощью PCA и восстанавливает его при вызове.

    Attributes
    ----------
    img_path_ : str
        Сохраняет путь к файлу изображения.
        Загружается изображение в формате jpg, имеющее форму (H, W, 3)
    n_components_ : int
        Количество основных компонентов, используемых для сжатия.
    mean_ : ndarray размера (W, 3)
        Среднее значение исходного изображения, используемого для центрирования.
    Y_ : ndarray размера (3, H, n_components_)
        Сжатые данные изображения после применения PCA.
    Vt_ : ndarray размера (3, n_components_, W)
        Топ n основных компонентов SVD.
    """

    def __init__(self, img_path: str, n_components: int) -> None:
        self.img_path_ = img_path
        self.n_components_ = n_components

        self._compress()

    def _compress(self) -> None:
        """        
        Сжатие изображения с использованием SVD разложения.

        Сначала изображение преобразуется в массив и центрируется. Затем применяется SVD
        для сжатия данных изображения. Этот метод устанавливает атрибуты Y_ и Vt_ .
        """
        with Image.open(self.img_path_) as img:
            img = np.array(img)

        self.mean_ = img.mean(axis=0)
        img = img - self.mean_
        img = np.moveaxis(img, 2, 0)

        _, _, Vt = np.linalg.svd(img)

        Vt_ = Vt[:, : self.n_components_]
        Y_ = np.matmul(img, Vt_.transpose(0, 2, 1))

        self.Y_ = Y_
        self.Vt_ = Vt_

    def __call__(self) -> PILImage:
        """        
        Восстанвление сжатого изображения.

        Когда вызывается экземпляр, он восстанавливает изображение из сжатых данных
        и возвращает его как объект PIL Image.

        Returns
        -------
        PILImage
            Восстановленное изображение после сжатия алгоритмом PCA.
        """
        img_ = np.matmul(self.Y_, self.Vt_)
        img_ = img_.transpose(1, 2, 0)
        img_ += self.mean_
        img_ = img_.clip(0, 255)
        img_ = img_.astype(np.uint8)
        return Image.fromarray(img_)

In [7]:
img_path = "mug.jpg"
n_components = 64

img_pca = ImagePCA(img_path, n_components)
img = img_pca()

img.save("reconstructed.jpg")